In [27]:
import sys
import requests
import csv
import json
import pandas as pd

Issue Crawler

In [28]:
def print_gap(num):
    for i in range(num): 
        print('')

In [29]:
def print_pattern():
    print('-' * 10)

In [30]:
def gen_api_url(filename):
    api_url = "https://api.github.com/repos/"
    with open(filename, "r") as file:
        file_data = json.load(file)      

    try:
        repo = file_data["repo_link"]
    except KeyError:
        print_gap(1)
        # print("Repo not stored.")
        repo = input("Enter Repo link: ")
        print_gap(1)

        # TODO: Add ability to store repo link
        # input("Do you want me to store repo? (Y/N): ")

    if len(repo.split('/')[-2:]) == 1:
        print('Invalid Url')
        return 'invalid'
    else:
        api_url += '/'.join(repo.split('/')[-2:])
        return api_url


In [31]:
## TODO: Add params. Only supports access_token now
def add_params_to_url(api_url, filename):
    with open(filename, "r") as file:
        file_data = json.load(file)   

    # try:
    #     access_token = file_data["access_token"]
    # except KeyError:
    #     print_pattern()
    #     # print("Access token not stored.")
    #     print("Access token required for private github repos, if public press enter(return), enter token otherwise.")
    #     print("Help on creating token - https://help.github.com/articles/creating-an-access-token-for-command-line-use/")
    #     print_pattern()
    #     print_gap(1)
    #     access_token = input("Enter Access Token: ")
    #     print_gap(1)

        # TODO: Add ability to store access_token
        # input("Do you want me to store access token? (Y/N): ")


    api_url += '/issues?state=closed'

    return api_url

Main

In [32]:
api_url = add_params_to_url(gen_api_url('repo_link.txt'), 'repo_link.txt')
print(api_url)
req = requests.get(api_url)
    # response = requests.get(api_url)
    # for issue in response.json():
        
    #     labels = issue['labels']
    #     for label in labels:
    #         print (label)
data = req.json()
    
# output = open('output.txt', 'w+')
# keys = data[0].keys()
# output.write(" ".join(keys))



fields = ['number', 'title', 'html_url']

other = ['assignee', 'creater']

with open("crawled_issue/data.csv", "w") as file:
    csv_file = csv.writer(file)
    cols = fields + other
    csv_file.writerow(cols)
    for item in data:
            row = [item[i] for i in fields]
            try:
                row += [item['assignee']['login']]
            except:


                row += [' ']
            row += [item['user']['login']]
            csv_file.writerow(row)

https://api.github.com/repos/umami-software/umami/issues?state=closed


In [33]:

query_parameters = {"downloadformat": "csv"}
response = requests.get(api_url, params=query_parameters)
with open("crawled_issue/report.json", mode="wb") as file:
    file.write(response.content)


JSON Parser

In [34]:
#read report.json file and return the json object
def read_report_json():
    with open('crawled_issue/report.json') as json_file:
        data = json.load(json_file)
        return data
    
data = read_report_json()



In [35]:
# Save to CSV
with open('crawled_issue/output.csv', 'w', newline='') as csvfile:
    fieldnames = ['Issue ID','Issue Body']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    cnt=0
    writer.writeheader()
    for item in data:
        if item['body'] is not None:
            writer.writerow({'Issue ID':item['id'],'Issue Body': item['body']})
            print(cnt)
            cnt+=1
    
#sort the csv file by Issue ID
df = pd.read_csv('crawled_issue/output.csv')
df.sort_values(by=['Issue ID'], inplace=True)
df.to_csv('crawled_issue/output.csv', index=False)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
